In [ ]:
test_samplefile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\test_sample.dat ',sep='\t',
                              nrows=30000,header=None,names=['样本id','广告id','创建时间','素材尺寸','广告行业id'
                                                           ,'商品类型','商品id','广告账户id','投放时间','人群定向'
                                                            ,'出价'])
test_sampledf = pd.DataFrame(test_samplefile)

In [ ]:
sampleadlist=test_sampledf.广告id.unique()

In [ ]:
# 筛选出历史曝光中样本id相关的曝光记录
exposureLogfile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\totalExposureLog.out ',sep='\t',
                              header=None,names=['请求id','请求时间','广告位id','用户id','曝光广告id'
                                                            ,'素材尺寸','出价bid','pctr','quality_ecpm','totalEcpm']
                              ,iterator=True,low_memory =False)
n=3000
temp=pd.DataFrame()
for i in range(n):
    try:
        print(i)
        user = exposureLogfile.get_chunk(100000)
        user.drop_duplicates(keep='first', inplace=True)
        user=user[user['曝光广告id'].isin(sampleadlist)]
        if (temp.empty):
            temp=user
        else:
            temp=temp.append(user,ignore_index=True)
    except StopIteration:
        break

In [ ]:
temp['请求时间']=temp['请求时间'].apply(lambda x:time.strftime("%Y%m%d",time.localtime(x))) 
temp.to_csv("E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\histroy_sample.csv",
            sep="\t",index=False,encoding="utf-8",header=None)

In [ ]:
temp=pd.DataFrame(pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\histroy_sample.csv ',sep='\t',
                              header=None,names=['请求id','请求时间','广告位id','用户id','广告id'
                                                            ,'素材尺寸','出价','pctr','quality_ecpm','totalEcpm']
                              ,low_memory =False)
                 )

In [ ]:
temp1=temp.groupby(['广告id','请求时间','素材尺寸','出价'])['请求id'].count().reset_index()
temp1.rename(columns={"请求id":"曝光次数"},inplace=True)

In [ ]:
#temp2=temp1.groupby(['广告id','出价'])['曝光次数'].median().reset_index()
temp2=temp1.groupby(['广告id','出价'])['曝光次数'].median().reset_index()
temp2.to_csv("E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\histroy_sample1.csv",
            sep="\t",index=False,encoding="utf-8")

In [ ]:
test_samplefile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\test_sample.dat ',sep='\t',
                              nrows=30000,header=None,names=['样本id','广告id','创建时间','素材尺寸','广告行业id'
                                                           ,'商品类型','商品id','广告账户id','投放时间','人群定向'
                                                            ,'出价'])
test_sampledf = pd.DataFrame(test_samplefile)

In [ ]:
test_sampledf['排名']=((test_sampledf['出价'].groupby(test_sampledf['广告id']).rank(ascending=1,method='dense'))).round(decimals=4)

In [ ]:
#test_sampledf=pd.merge(test_sampledf,temp2,how="left",left_on=['广告id','出价'],right_on=['广告id','出价'])
test_sampledf=pd.merge(test_sampledf,temp2,how="left",left_on=['广告id'],right_on=['广告id'])
test_sampledf.to_csv("E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\test_sample_with_his.csv",
            sep="\t",index=False,encoding="utf-8")

In [ ]:
# 根据历史广告获取中间值曝光次数，不考虑报价，将样本中同一id的排名为5的作为基准，其他的按照  基准*排名/5计算，对于新广告默认为4
# 在excel中操作的

In [ ]:
# 得分为84分